In [69]:
import spacy
from spacy.symbols import ORTH, LEMMA, POS
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")


<function spacy.pipeline.functions.merge_noun_chunks(doc: spacy.tokens.doc.Doc) -> spacy.tokens.doc.Doc>

In [46]:
doc = nlp(open('../assets/data/raw/AbrahamLincoln.txt').read())

def born(sent):
    if 'born' not in sent.root.text :
        return None

    subj = [c for c in sent.root.children if c.dep_ == 'nsubjpass']
    places = [e for e in sent.ents if e.label_ == 'GPE' and e.root.head.tag_ =='IN'] 
    dates = [e for e in sent.ents if e.label_ == 'DATE' and e.root.head.tag_ =='IN']
    person = subj[0].text if subj else ''
    date = dates[0].text if dates else ''
    place = places[0].text if places else ''
    if person:
        return person, date, place
        
for sent in doc.sents:
    t = born(sent)
    if t:
        print (t)

('Abraham Lincoln', 'February 12, 1809', 'Hodgenville')
('Robert Todd Lincoln', '1843', '')
('Willie" Lincoln', 'December 21, 1850', '')
("The Lincolns' fourth son", 'April 4, 1853', '')


In [89]:
doc = nlp(open('../assets/data/raw/SteveJobs.txt').read())

def founded_active(sent):
    if 'founded' not in sent.root.text :
        return None
    
    subj = [c for c in sent.root.children if c.dep_ == 'nsubj']
    obj = [c for c in sent.root.children if c.dep_ == 'dobj']

    dates = [e for e in sent.ents if e.label_ == 'DATE' and e.root.head.tag_ =='IN']
    places = [e for e in sent.ents if e.label_ == 'GPE' and e.root.head.tag_ =='IN'] 

    org = obj[0] if obj else ''
    date = dates[0] if dates else ''
    place = places[0] if places else ''

    if org:
        return org, date, place

def founded_passive(sent):
    if ('founded' not in sent.text.lower()) and ('established' not in sent.text.lower()):
        return None
    obj = [e for e in sent.ents if e.label_ == 'ORG']# and e.root.head.text == 'founded']

    dates = [e for e in sent.ents if e.label_ == 'DATE' and e.root.head.tag_ =='IN']
    places = [e for e in sent.ents if e.label_ == 'GPE' and e.root.head.tag_ =='IN'] 

    org = obj[0] if obj else ''
    date = dates[0] if dates else ''
    place = places[0] if places else ''

    if org and date:
        return org, date, place

def founded(sent):
    t = founded_active(sent)
    if not t:
        t = founded_passive(sent)
    return t
for sent in doc.sents:
    t = founded(sent)
    if t:
        print (sent)
        print (t)



Jobs and Wozniak co-founded Apple in 1976 to sell Wozniak's Apple I personal computer.
(founded Apple, 1976, '')
Jobs, Wozniak, and Ronald Wayne founded Apple Computer (now called Apple Inc.) in the garage of Jobs's Los Altos home on Crist Drive.
(Apple Computer, '', '')
Following his resignation from Apple in 1985, Jobs founded NeXT Inc. with $7 million.
(NeXT Inc., 1985, '')


In [91]:
def born_template(sent):
    relations = []
    t = born(sent)
    if t:
        return t
    
    t = founded(sent)
    if t:
        return t
doc = nlp(open('../assets/data/raw/Washington_DC.txt').read())
for sent in doc.sents:
    t = born_template(sent)
    if t:
        print (sent)
        print (t)

Founded in 1789, Georgetown University features a mix of Romanesque and Gothic Revival architecture.
(Georgetown University, 1789, '')
In addition to the District's own Metropolitan Police Department, many federal law enforcement agencies have jurisdiction in the city as well – most visibly the U.S. Park Police, founded in 1791.
(the District's own Metropolitan Police Department, 1791, '')
The Marine Barracks near Capitol Hill houses the United States Marine Band; founded in 1798, it is the country's oldest professional musical organization.
(The Marine Barracks, 1798, '')
Founded in 1925, the United States Navy Band has its headquarters at the Washington Navy Yard and performs at official events and public concerts around the city.
(the Washington Navy Yard, 1925, '')
Founded in 1950, Arena Stage achieved national attention and spurred growth in the city's independent theater movement that now includes organizations such as the Shakespeare Theatre Company, Woolly Mammoth Theatre Compa